<a href="https://colab.research.google.com/github/danielbauer1979/FI830/blob/main/FI830_HW7_S24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install QuantLib

In [2]:
import yfinance as yf
import QuantLib as ql
import numpy as np
import statistics as st
from scipy.optimize import newton
from scipy.stats import multivariate_normal
from scipy.stats import norm

## Part a Calibration \& Estimation

In [3]:
# step 1: obtain S&P 500 and VIX data for the past two years
sp500 = yf.download('^SPX', start='2022-04-17',end='2024-04-17')['Close'].values
vix = yf.download('^VIX', start='2022-04-17',end='2024-04-17')['Close'].values/100
vt = vix**2

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
# asssume kappa=2.5, r from one-year constant maturity treasury rate
kappa = 2.5; r = 0.0517; delta_t = 2/len(sp500)

In [5]:
# step 2: define a function for Xt where the parameters are theta and t
def Xt(t,theta,kappa=kappa):
  return ((vt[t+1]-vt[t])-kappa*(theta-vt[t])*delta_t)/(np.sqrt(vt[t])*np.sqrt(delta_t))
def avgXt(theta,kappa=kappa):
  Xarr = [Xt(t,theta,kappa) for t in range(len(vt)-1)]
  return np.mean(Xarr)

In [6]:
theta_hat = newton(lambda theta: avgXt(theta),1)
theta_hat

0.04603961112024731

In [7]:
# plug theta_hat into Xt function to get a list of Xt
# use standard deviation of Xt list to get sigma_v
Xt_arr = [Xt(t,theta_hat) for t in range(len(vt)-1)]
sigma_v_hat = st.stdev(Xt_arr)
sigma_v_hat

0.4114518663812112

In [8]:
# we know r, kappa, theta, sigma_v
# also need mu, rho
# repeat step 2 to get mu
def Yt(t,mu):
  return (sp500[t+1]-sp500[t]-sp500[t]*mu*delta_t)/(sp500[t]*np.sqrt(vt[t])*np.sqrt(delta_t))

def avgYt(mu):
  Yarr = [Yt(t,mu) for t in range(len(sp500)-1)]
  return np.mean(Yarr)

mu_hat = newton(lambda mu: avgYt(mu),1)
mu_hat

0.09901852004284824

In [9]:
# plug mu_hat into Yt and get a list of Yt
Yt_arr = [Yt(t,mu_hat) for t in range(len(sp500)-1)]

# remaining parameter to estimate: rho
# calcuate the correlation between Xt and Yt
rho_hat = np.corrcoef(Xt_arr,Yt_arr)[0,1]
rho_hat

-0.7184238679866422

## Part b Heston Model (Analytical)

In [10]:
# Fetch options data
options = yf.Ticker("^SPX")
expiry_date = "2025-04-17"
option_chain = options.option_chain(expiry_date)
call_df = option_chain.calls

call_df[call_df['strike']==5000]['lastPrice'] # somewhat a close match to the 1-year at-the-money call option, but not exact

14    486.51
Name: lastPrice, dtype: float64

In [11]:
# current levels of S&P500 and VIX
S0 = sp500[-1]; v_0 = vt[-1]

In [12]:
today = ql.Date(17,4,2024)
day_count = ql.Actual365Fixed()
strike = S0
maturity = ql.Date(17,4,2025)
initial_value = ql.QuoteHandle(ql.SimpleQuote(S0))
discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today,r,day_count))
dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(today,0,day_count))
heston_process = ql.HestonProcess(discount_curve,dividend_yield,initial_value,v_0,kappa,theta_hat,sigma_v_hat,rho_hat)
call_payoff = ql.PlainVanillaPayoff(ql.Option.Call,strike)
call_exercise = ql.EuropeanExercise(maturity)
option = ql.VanillaOption(call_payoff,call_exercise)
engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process),0.001,1000)
option.setPricingEngine(engine)
price = option.NPV()
price

534.2505608292646

## Part c Heston Model (Monte Carlo)

In [13]:
#MC parameters
N = 10000
delta_t_mc = 1/500

In [14]:
def MCHeston(S0,K,r,T,kappa,theta,sigma_v,v0,rho,N,delta_t):
  X0 = np.log(S0)
  C_0 = [0] * N
  for n in range(N):
    d = int(T/delta_t+1)
    Xt = [X0] * d
    v_t = [v0] * d
    epsilon = multivariate_normal.rvs(mean=[0,0],cov=[[1,rho],[rho,1]],size=d-1)
    for i in range(1,d):
      Xt[i] = Xt[i-1] + (r-0.5*v_t[i-1])*delta_t + np.sqrt(v_t[i-1]*delta_t)*epsilon[i-1,0]
      v_t[i] = max(0,v_t[i-1] + kappa*(theta-v_t[i-1])*delta_t+sigma_v*np.sqrt(v_t[i-1]*delta_t)*epsilon[i-1,1])
    C_0[n] = max(0,np.exp(Xt[-1])-K)*np.exp(-r*T)
  return(np.average(C_0))

In [15]:
np.random.seed(12)
mc_call = MCHeston(S0,S0,r,1,kappa,theta_hat,sigma_v_hat,v_0,rho_hat,N,delta_t_mc)
mc_call # already very close to the option price from analytical heston model

538.0463371400278

## Part d Parameter Tuning

Experiment with kappa and re-estimate theta

In [18]:
kappas = [2,2,3,3]
thetas = [0.04, 0.05, 0.04, 0.05]

In [19]:
MC_prices = []
for i in range(len(kappas)):
  np.random.seed(12)
  MC_prices += [MCHeston(S0,S0,r,1,kappas[i],thetas[i],sigma_v_hat,v_0,rho_hat,N,delta_t_mc)]
MC_prices

[516.1434729974186, 542.6906326959385, 523.1357658376268, 554.3498940547628]

In [20]:
kappas = [1.5,1.5,2,2.5]
thetas = [0.04,0.03, 0.03, 0.03]

In [21]:
MC_prices = []
for i in range(len(kappas)):
  np.random.seed(12)
  MC_prices += [MCHeston(S0,S0,r,1,kappas[i],thetas[i],sigma_v_hat,v_0,rho_hat,N,delta_t_mc)]
MC_prices

[510.80493654211693, 486.17187841001555, 487.71757850452343, 488.5179337556684]

In [25]:
theta = 0.03
kappa = 1.5

# Part e Greeks

In [55]:
# via quantlib
today = ql.Date(17,4,2024)
day_count = ql.Actual365Fixed()
strike = S0
maturity = ql.Date(17,4,2025)
initial_value = ql.QuoteHandle(ql.SimpleQuote(S0))
discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today,r,day_count))
dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(today,0,day_count))
heston_process = ql.HestonProcess(discount_curve,dividend_yield,initial_value,v_0,kappa,theta,sigma_v_hat,rho_hat)
engine_price = ql.AnalyticHestonEngine(ql.HestonModel(heston_process),0.001,1000)
option.setPricingEngine(engine_price)
price = option.NPV()

# Calculate option delta
delta_s0 = 0.01
S0_up = S0 + delta_s0
initial_value_up = ql.QuoteHandle(ql.SimpleQuote(S0_up))
heston_process_price_up = ql.HestonProcess(discount_curve,dividend_yield,initial_value_up,v_0,kappa,theta,sigma_v_hat,rho_hat)
engine_price_up = ql.AnalyticHestonEngine(ql.HestonModel(heston_process_price_up),0.001,1000)
option.setPricingEngine(engine_price_up)
option_price_up = option.NPV()
delta = (option_price_up - price) / delta_s0

# Calculate option vega
delta_v0 = 0.001*0.001
vol_up = v_0 + delta_v0
heston_process_vol_up = ql.HestonProcess(discount_curve,dividend_yield,initial_value,vol_up,kappa,theta,sigma_v_hat,rho_hat)
engine_vol_up = ql.AnalyticHestonEngine(ql.HestonModel(heston_process_vol_up),0.001,1000)
option.setPricingEngine(engine_vol_up)
option_vol_up = option.NPV()
vega = (option_vol_up - price) / delta_v0

print("Option Price:", price)
print("Delta:", delta)
print("Vega:", vega)

Option Price: 482.02345377195326
Delta: 0.7433772215392764
Vega: 2565.134532119373


In [56]:
# via Monte Carlo finite difference
def MCHestonGreeks(S0,K,r,T,kappa,theta,sigma_v,v0,rho,N,delta_t,delta_S0,delta_v0):
  X0 = np.log(S0)
  X0_dS0 = np.log(S0+delta_S0)
  Delta_0 = [0] * N
  Vega_0 = [0] * N
  for n in range(N):
    d = int(T/delta_t+1)
    Xt = [X0] * d
    Xt_dS0 = [X0_dS0] * d
    Xt_dv0 = [X0] * d
    v_t = [v0] * d
    v_t_1 = [v0+delta_v0] * d
    epsilon = multivariate_normal.rvs(mean=[0,0],cov=[[1,rho],[rho,1]],size=d-1)
    for i in range(1,d):
      Xt[i] = Xt[i-1] + (r-0.5*v_t[i-1])*delta_t + np.sqrt(v_t[i-1]*delta_t)*epsilon[i-1,0]
      Xt_dS0[i] = Xt_dS0[i-1] + (r-0.5*v_t[i-1])*delta_t + np.sqrt(v_t[i-1]*delta_t)*epsilon[i-1,0]
      Xt_dv0[i] = Xt_dv0[i-1] + (r-0.5*v_t_1[i-1])*delta_t + np.sqrt(v_t_1[i-1]*delta_t)*epsilon[i-1,0]
      v_t[i] = max(0,v_t[i-1] + kappa*(theta-v_t[i-1])*delta_t+sigma_v*np.sqrt(v_t[i-1]*delta_t)*epsilon[i-1,1])
      v_t_1[i] = max(0,v_t_1[i-1] + kappa*(theta-v_t_1[i-1])*delta_t+sigma_v*np.sqrt(v_t_1[i-1]*delta_t)*epsilon[i-1,1])

    C_0 = max(np.exp(Xt[-1])-K,0)*np.exp(-r*T)
    C_0_dS0 = max(np.exp(Xt_dS0[-1])-K,0)*np.exp(-r*T)
    C_0_dv0 = max(np.exp(Xt_dv0[-1])-K,0)*np.exp(-r*T)
    Delta_0[n] = (C_0_dS0-C_0)/delta_S0
    Vega_0[n] = (C_0_dv0-C_0)/delta_v0

  return(np.average(Delta_0),np.average(Vega_0))

In [59]:
delta_S0 = 0.001
delta_v0 = 0.001*0.001
np.random.seed(12)
delta_mc, vega_mc = MCHestonGreeks(S0,S0,r,1,kappa,theta,sigma_v_hat,v_0,rho_hat,N,delta_t_mc,delta_S0,delta_v0)

print("Option Price:", MC_prices[1])
print("Delta:", delta_mc)
print("Vega:", vega_mc)

Option Price: 486.17187841001555
Delta: 0.742646836717334
Vega: 2626.7666356386735
